3. Positive, Negative 말고 Neutral 데이터도 추가해서 ML모델을 만들고 성능을 분석하기 (Pos, Neg만 썼을 때와 성능 차이 있나?)

**Training**

In [2]:
import pandas as pd
df_training = pd.read_csv('twitter_training.csv')
df_new_t = df_training[~df_training.iloc[:,2].isin(['Irrelevant'])]

categories_t = df_new_t.iloc[:, 2].tolist()
reviews_t = df_new_t.iloc[:, 3].tolist()
reviews_t = [str(doc) if pd.notna(doc) else '' for doc in reviews_t]

print(set(categories_t))

{'Positive', 'Negative', 'Neutral'}


In [3]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

english_stops = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')

#리뷰 전처리
def preprocess_reviews(reviews):
    processed_reviews = []
    for review in reviews:
        if isinstance(review, str):  # 리뷰가 문자열인지 
            tokens = tokenizer.tokenize(review.lower())
            filtered_words = [word for word in tokens if word not in english_stops]
            processed_reviews.append(' '.join(filtered_words))
        else:
            processed_reviews.append('') 
    return processed_reviews

# 리뷰 전처리 실행
cleaned_reviews_t = preprocess_reviews(reviews_t)

**Validation**

In [4]:
import pandas as pd
df_validation = pd.read_csv('twitter_validation.csv')
df_new_v = df_validation[~df_validation.iloc[:,2].isin(['Irrelevant'])]

categories_v = df_new_v.iloc[:, 2].tolist()
reviews_v = df_new_v.iloc[:, 3].tolist()
reviews_v = [str(doc) if pd.notna(doc) else '' for doc in reviews_v]
print(set(categories_v))

{'Positive', 'Negative', 'Neutral'}


In [5]:
cleaned_reviews_v = preprocess_reviews(reviews_v)

In [6]:
# Naive Bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score

# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(max_features=5000) 
train_features = tfidf_vectorizer.fit_transform(reviews_t)
test_features = tfidf_vectorizer.transform(reviews_v)

model = MultinomialNB()  
model.fit(train_features, categories_t)

# 예측
train_pred = model.predict(train_features)
val_pred = model.predict(test_features)

# 성능 평가
train_accuracy = accuracy_score(categories_t, train_pred)
val_accuracy = accuracy_score(categories_v, val_pred)
train_f1 = f1_score(categories_t, train_pred, average='weighted')
val_f1 = f1_score(categories_v, val_pred, average='weighted')


print(f"Training Accuracy: {train_accuracy}")
print(f"Validation Accuracy: {val_accuracy}")
print(f"Training F1 Score: {train_f1}")
print(f"Validation F1 Score: {val_f1}")

Training Accuracy: 0.7428149973253797
Validation Accuracy: 0.7596618357487923
Training F1 Score: 0.7385551269988125
Validation F1 Score: 0.7568696623589081


**결과**

앞에서 이진 분류(Positive/Negative)에서 성능이 다음과 같았다.

Training Accuracy: 0.8681
Validation Accuracy: 0.8932, 
Training F1 Score: 0.8679
Validation F1 Score: 0.8932

다중 분류(Positive/Negative/Neutral)에서 성능이 다음과 같았다.

Training Accuracy: 0.7428
Validation Accuracy: 0.7597
Training F1 Score: 0.7386
Validation F1 Score: 0.7569

그 결과 다중 분류에서 성능이 저하된 걸 볼 수 있다. 

이진 분류에서는 두 감성 클래스 간의 차이가 명확하고 구분이 쉬운 반면, 다중 분류에 중립 클래스가 추가되면서 각 감성 간의 구분이 복잡해진 것 같다. 

또한 Naive Bayes는 각 특징이 독립적이라는 가정을 바탕으로 동작한다. 감성 분석에서 중립 클래스는 긍정 및 부정 감성이 혼합된 특징을 가져서 독립성 가정이 성능에 부정적 영향을 미친 것으로 보인다.
